# Retrieval-Augmented Generation (RAG)

In [ ]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

Create a static calorie table that we can use as a tool:

In [ ]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook


In [ ]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

In [ ]:
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [ ]:
# calorie_lookup_tool('bananas')

In [ ]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
)

In [ ]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)